 ## Gộp folder chứa file HDF5 thành folder mới chứa toàn bộ file hdf5

In [1]:
import os
import shutil

In [3]:

# Thư mục gốc chứa các thư mục con
root_dir = r"E:\co2_gosat\data\gosat_tar"
# Thư mục đích để lưu toàn bộ file HDF5 đã gộp
output_dir = r"E:\co2_gosat\data\gosat_hdf5"
# Tạo thư mục đích nếu chưa có
os.makedirs(output_dir, exist_ok=True)


In [4]:

# Duyệt qua tất cả các thư mục và file bên trong root_dir
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        if file.endswith(".h5") or file.endswith(".hdf5"):
            src_path = os.path.join(subdir, file)
            dst_path = os.path.join(output_dir, file)

            # Nếu file trùng tên, thêm số để tránh ghi đè
            base, ext = os.path.splitext(file)
            counter = 1
            while os.path.exists(dst_path):
                dst_path = os.path.join(output_dir, f"{base}_{counter}{ext}")
                counter += 1

            shutil.copy2(src_path, dst_path)

print("✅ Đã gộp xong tất cả file .h5 vào thư mục:", output_dir)

✅ Đã gộp xong tất cả file .h5 vào thư mục: E:\co2_gosat\data\gosat_hdf5


## Cấu trúc file hdf5


In [6]:
import h5py

file_path = r"E:\co2_gosat\data\gosat_hdf5\GOSATTFTS20200102_02C01SV0305R200102GU000.h5"

def print_structure(name):
    print(name)

with h5py.File(file_path, 'r') as f:
    print("📂 Cấu trúc file HDF5:")
    f.visit(print_structure)

📂 Cấu trúc file HDF5:
Data
Data/auxiliaryParameter
Data/auxiliaryParameter/aerosolOpticalThickness
Data/auxiliaryParameter/cirrusOpticalThickness
Data/auxiliaryParameter/cirrusTopPressure
Data/auxiliaryParameter/dryAirTotalColumn
Data/auxiliaryParameter/surfaceAlbedo
Data/auxiliaryParameter/surfacePressure
Data/biasCorrectionInfo
Data/biasCorrectionInfo/biasCorrectionCoefficient
Data/biasCorrectionInfo/biasCorrectionCoefficient2
Data/biasCorrectionInfo/biasCorrectionFormula
Data/biasCorrectionInfo/biasCorrectionParameterAverage
Data/biasCorrectionInfo/biasCorrectionParameterAverage2
Data/biasCorrectionInfo/usedTCCONdata
Data/biasCorrectionInfo/usedTCCONdata/conditionMatchup
Data/biasCorrectionInfo/usedTCCONdata/referencedTCCONdataSite
Data/biasCorrectionInfo/usedTCCONdata/referencedTCCONdataSite/endDate
Data/biasCorrectionInfo/usedTCCONdata/referencedTCCONdataSite/name
Data/biasCorrectionInfo/usedTCCONdata/referencedTCCONdataSite/numMatchup
Data/biasCorrectionInfo/usedTCCONdata/referen

## Xuất file .csv chứa thông tin cần thiết

In [7]:
import os
import h5py
import numpy as np
import pandas as pd
from tqdm import tqdm

In [11]:

# Đường dẫn tới thư mục chứa các file .h5 đã được gộp
folder_path = r"E:\co2_gosat\data\gosat_hdf5"
# Đường dẫn lưu file CSV đầu ra
output_csv = r"E:\co2_gosat\data\gosat_csv\gosat_all_2020.csv"
# # Tạo thư mục đích nếu chưa có
# os.makedirs(output_csv, exist_ok=True)



In [12]:
# Ánh xạ biến cần lấy
dataset_paths = {
    "lat": "Data/geolocation/latitude",
    "lon": "Data/geolocation/longitude",
    "sza": "Data/geolocation/solarZenith",
    "vza": "Data/geolocation/satelliteZenith",
    "saa": "Data/geolocation/solarAzimuth",
    "vaa": "Data/geolocation/satelliteAzimuth",
    "xco2": "Data/mixingRatio/XCO2",
    "xco2_sigma": "Data/mixingRatio/XCO2BiasCorrectedError",
    "xco2_qual": "Data/retrievalQuality/totalPostScreeningResult",
    "time": "scanAttribute/time"
}

In [14]:
def read_h5_file(file_path):
    data_dict = {}
    try:
        with h5py.File(file_path, 'r') as f:
            for key, path in dataset_paths.items():
                if path in f:
                    data = f[path][()]
                    data_dict[key] = data
                else:
                    print(f"⚠️ Không tìm thấy '{path}' trong {file_path}")
                    return None

            # Lấy ngày từ tên file (VD: GOSATTFTS20200102... -> 2020-01-02)
            filename = os.path.basename(file_path)
            date_str = filename[9:17]  # GOSATTFTS20200101_... -> "20200101"
            date = pd.to_datetime(date_str, format="%Y%m%d")
            data_dict["date"] = np.full(data_dict["xco2"].shape, date)  # ✅

            return pd.DataFrame(data_dict)

    except Exception as e:
        print(f"❌ Lỗi đọc {file_path}: {e}")
        return None

In [15]:
# Tìm tất cả các file .h5
h5_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(".h5")]

# Gộp dữ liệu
all_dfs = []
for file in tqdm(h5_files, desc="📦 Đang xử lý các file"):
    df = read_h5_file(file)
    if df is not None:
        all_dfs.append(df)

# Gộp tất cả lại
if all_dfs:
    final_df = pd.concat(all_dfs, ignore_index=True)
    final_df.to_csv(output_csv, index=False)
    print(f"\n✅ Đã lưu file CSV: {output_csv}")
else:
    print("\n⚠️ Không có file nào được xử lý thành công.")

📦 Đang xử lý các file: 100%|██████████| 389/389 [00:23<00:00, 16.89it/s]



✅ Đã lưu file CSV: E:\co2_gosat\data\gosat_csv\gosat_all_2020.csv


## Xuất dữ liệu CO2 từ file gosat_all lọc theo ranh giới VietNam sang file gosat_vietnam

In [16]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


In [17]:

input_csv_path = r"E:\co2_gosat\data\gosat_csv\gosat_all_2020.csv"
input_shapefile_path = r"E:\co2_gosat\VNM_adm\gadm41_VNM_0.shp"  # Biên giới Việt Nam
output_csv_path = r"E:\co2_gosat\data\gosat_csv\gosat_vietnam_2020.csv"  # File kết quả sau khi lọc

# Đọc file CSV
df = pd.read_csv(input_csv_path)

# Đọc shapefile
gdf_vietnam = gpd.read_file(input_shapefile_path)

# Đảm bảo hệ tọa độ là EPSG:4326
if gdf_vietnam.crs != "EPSG:4326":
    gdf_vietnam = gdf_vietnam.to_crs(epsg=4326)

# Chuyển DataFrame thành GeoDataFrame với cột geometry từ lat/lon
geometry = [Point(xy) for xy in zip(df["lon"], df["lat"])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# ====== LỌC DỮ LIỆU ======
# Lọc sơ bộ theo bounding box của Việt Nam
min_lon, min_lat, max_lon, max_lat = gdf_vietnam.total_bounds
gdf = gdf[
    (gdf["lon"] >= min_lon) & (gdf["lon"] <= max_lon) &
    (gdf["lat"] >= min_lat) & (gdf["lat"] <= max_lat)
]

# Dùng spatial join để lấy điểm nằm trong vùng đất liền
gdf_land = gpd.sjoin(gdf, gdf_vietnam, predicate="within", how="inner")

# ====== GHI RA FILE ======
# Ghi ra file CSV, loại bỏ cột phụ
gdf_land.drop(columns=["geometry", "index_right"], errors="ignore").to_csv(
    output_csv_path, index=False
)

print(f"✅ Đã xuất file thành công: {output_csv_path}")


✅ Đã xuất file thành công: E:\co2_gosat\data\gosat_csv\gosat_vietnam_2020.csv
